In [ ]:
# Basic plots
%matplotlib inline
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8]

# Common imports and settings
import os, sys
os.environ['USE_PYGEOS'] = '0'
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr

# Datacube
import datacube
from datacube.utils.rio import configure_s3_access
from datacube.utils import masking
from datacube.utils.cog import write_cog
# https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools
from dea_tools.plotting import display_map, rgb
from dea_tools.datahandling import mostcommon_crs

# EASI defaults
easinotebooksrepo = '/home/jovyan/easi-notebooks'
if easinotebooksrepo not in sys.path: sys.path.append(easinotebooksrepo)
from easi_tools import EasiDefaults, xarray_object_size, notebook_utils

In [ ]:
# Data tools
import numpy as np
from datetime import datetime

# Datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-algo/blob/main/odc/algo/_masking.py
from odc.algo import xr_reproject   # https://github.com/opendatacube/odc-algo/blob/main/odc/algo/_warp.py
from datacube.utils.geometry import GeoBox, box  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/geometry/_base.py

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# import geoviews as gv
# from holoviews.operation.datashader import rasterize
hv.extension('bokeh', logo=False)

## Dask cluster

There are two dask cluster options available:

- *LocalCluster* runs on the JupyterLab node so is limited to the amount of memory available (depending on the resource type selected at login). Dask does allow for using more than the available memory by swapping data to disk but there are practical limits to this. Good for testing and demonstrating smaller workflows.
- *Dask Gateway* creates a *dask scheduler* and a set of *dask workers* that run in their own pods. Workers may still fail due to out-of-memory but if so the scheduler will attempt to replace the worker and reassign its tasks to other workers. Good for developing and running larger or quicker workflows.

For more information see the set of notebooks at https://github.com/csiro-easi/easi-notebooks/tree/main/tutorials/dask.

Here I have added the gateway option. You already had the LocalCluster code.

In [ ]:
# Dask gateway

cluster, client = notebook_utils.initialize_dask(use_gateway=True, workers=(1,10))
client

In [ ]:
# Local cluster

## Can also use the initialize_dask() helper function.
## This function will attempt to re-use an existing LocalCluster if one is found.
## cluster, client = notebook_utils.initialize_dask(use_gateway=False, workers=(1,4))

# from dask.distributed import Client, LocalCluster

# cluster = LocalCluster(n_workers=2, threads_per_worker=4)
# # cluster.scale(n=2, memory="14GiB")
# cluster.scale(n=8, memory="6GiB")
# client = Client(cluster)
# display(client)

# dashboard_address = notebook_utils.localcluster_dashboard(client=client,server=easi.hub)
# display(dashboard_address)

## Initialise Datacube

In [ ]:
easi = EasiDefaults()

family = 'sentinel-2'
product = easi.product(family)
display(Markdown(f'Default {family} product for "{easi.name}": [{product}]({easi.explorer}/products/{product})'))

In [ ]:
dc = datacube.Datacube()

# Access AWS "requester-pays" buckets
# This is necessary for reading data from most third-party AWS S3 buckets such as for Landsat and Sentinel-2
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client);

## Read and verify the training data

In [ ]:
from utils import load_data_geo
# import geopandas as gpd
# from deafrica_tools.areaofinterest import define_area
# from datacube.utils.geometry import Geometry
# import xarray as xr
train_path = "train/Soc Trang_Traning.shp"
train = load_data_geo(train_path)
train.head()

In [ ]:
train1 = train.to_crs('EPSG:4326')

In [ ]:
train1.head().explore(column="Name", legend=True)

## Proposed workflow

1. Get bounding polygon for all training data points
1. Load datacube (dc.load with dask) for bounding polygon (and all times when you're ready to try that)
    - Consider also remapping S2 data to lat/lon projection (e.g., epsg:4326) - may not be necessary
1. Apply the S2 masking, scale, offset
1. Calculate NDVI (still in dask so its a "virtual" on-demand calculation)
1. Use xarray.persist() to pre-calculate NDVI for all pixels in our bounding polygon
   - Its usually more efficient to read and process all pixels than process each training point
1. For idx, point in train.iterrows():
    -  Get points from xarray (dask)
       - May need to convert (point lat/lon to S2 UTM) or (dc.load into epsg:4326)
       - Xarray data is in S2 UTM project (output_crs, resolution)
       - Point data is in epsg:4326 (train.crs)
    -  Store the loaded point data in the dictionary with a key based on the point index

In [ ]:
from deafrica_tools.bandindices import calculate_indices
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Vietnam
min_longitude, max_longitude = (105.5, 106.4)
min_latitude, max_latitude = (9.2, 10.0)
min_date = '2021-12-01' # 2021-11-01
max_date = '2022-01-01' # 2022-01-01
product = 's2_l2a'

query1 = {
    'product': product,                     # Product name
    'x': (min_longitude, max_longitude),    # "x" axis bounds
    'y': (min_latitude, max_latitude),      # "y" axis bounds
    'time': (min_date, max_date),           # Any parsable date strings
}

# Most common CRS
native_crs = notebook_utils.mostcommon_crs(dc, query1)

query1.update({
    'measurements': ['blue', 'green', 'red', 'nir', 'scl'],  # Selected measurement bands
    'output_crs': native_crs,               # EPSG code
    'resolution': (-10, 10),                # Target resolution
    'group_by': 'solar_day',                # Scene ordering
    'dask_chunks': {'x': 3310, 'y': 3000},  # Dask chunks
})


## Dask loading and chunking

The main tuning options for _dask_ are:

- The data _chunk_ size.
- Using _.persist()_ to pre-calculate intermediate stages and _.compute()_ to finalise the result back to your Jupyter session.

Here we can try different chunk shapes. A different chunk shape means that blocks of data get copied and calculated in different size aggregates, which can mean improved efficiency depending on the tasks being applied. We can change the chunk shape later if our algorithm will benefit from it.

In the `xarray.Dataset` graphic below, select one of the "stacked cylinder" icons on the right of a Data variable row. This shows a graphic of the array. Edit the `dask_chunks` values in the `query1.update()` function above. The target parameters for good general use are:

- Chunk size about 20 MB. It can be larger if need be but try not to it be too small.
- Number of chunks in the "Dask graph". 100s is good, 1000s is fine, 10000s is probably getting too many. Its a balance or trade-off between chunk size and the number of chunks.
- A low mod(dim-length, chunck-length) in each dimensionLimit small fractional chunks, thats is mod(dim-length, chunck-length) in each dimension.
- Square chunks and binary multiple sizes are often used but are not as important as the other three.

In [ ]:
# x = 3310
# y = 3000
# print(f"{9902.0/x} rem {9902 % x}")
# print(f"{8874.0/y} rem {8874 % y}")

In [ ]:
# Load data
data = dc.load(**query1)

notebook_utils.heading(notebook_utils.xarray_object_size(data))
display(data)

# Calculate valid (not nodata) masks for each layer
valid_mask = masking.valid_data_mask(data)
notebook_utils.heading('Valid data masks for each variable')
display(valid_mask)

In [ ]:
# Change variable name for dev
data1 = dc.load(**query1)

## Masking and scaling

The scale factor and offset need to manually applied to the arrays (the ODC chooses not make an assumption about these).

The Sentinel-2 L2A "SCL" band contains quality flags and information. These flags are distrete values (each pixel will have one of these values. In ODC we can use the `enum_to_bool()` function to create boolean mask from a combination of flag values.

In [ ]:
# Get the scale factor and offset from the measurement metadata

measurement_info = dc.list_measurements().loc[query1['product']].loc[query1['measurements']]  # Pandas dataframe
display(measurement_info)

# The "SCL" band contains quality flags and information. The details can also be found in the metadata.

flag_name = 'scl'
flag_desc = masking.describe_variable_flags(data[flag_name])  # Pandas dataframe
display(flag_desc)
display(flag_desc.loc['qa'].values[1])

In [ ]:
# Create a "data quality" Mask layer

flags_def = flag_desc.loc['qa'].values[1]
good_pixel_flags = [flags_def[str(i)] for i in [4, 5]]  # To pass strings to enum_to_bool()

# enum_to_bool calculates the pixel-wise "or" of each set of pixels given by good_pixel_flags
# 1 = good data
# 0 = "bad" data

good_pixel_mask = enum_to_bool(data[flag_name], good_pixel_flags)  # -> DataArray
# display(good_pixel_mask)  # Type: bool

### Sentinel-2 scaling and offset changes

ESA has undertaken a reprocessing of the Sentinel-2 L2A product, which includes a change to offset value used to convert digital numbers (in file) to scientific values (reflectances). ESA's reprocessing is flowing through to the [AWS open data repository of S2 L2A](https://registry.opendata.aws/sentinel-2/) but while this stabilises we may see inconsistencies in the COG files (encoded DNs) vs the STAC metadata (whether offset has been applied or not).

**TL;DR**: DN values have different definitions with different processing baselines!

[L2A algorithm and products](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/level-2a-algorithms-products): Starting with the PB 04.00 (25th January 2022), the dynamic range of the Level-2A products is shifted by a band-dependent constant: BOA_ADD_OFFSET. This offset will allow encoding negative surface reflectances that may occur over very dark surfaces

```
L2A_SRi = (L2A_DNi + BOA_ADD_OFFSETi) / QUANTIFICATION_VALUEi

QUANTIFICATION_VALUEi = 10000
BOA_ADD_OFFSETi = -1000

refl = (dn -1000) / 10000
refl = dn/10000 - 1000/10000
refl = dn * 0.0001 - 0.1   # These are the values in the product definition, https://explorer.asia.easi-eo.solutions/products/s2_l2a.odc-product.yaml
```

**What can we do about?** We need to add a check for a `earthsearch:boa_offset_applied` property in each dataset's metadata, and adjust the `offset` value accordingly. [Example per scene workflow](https://github.com/Element84/earth-search/issues/23#issuecomment-1834674853).

In [ ]:
# Apply valid mask (calculated above) and good pixel mask with scale and offset for each data layer and merge the results
# Optional - use .persist() on each layer or the result dataset

scale = 0.0001
offset = 0  # Assumes earthsearch:boa_offset_applied = True (else offset = -0.1)

data_layer_names = [x for x in data.data_vars if x != 'scl']

rs = []
for layer_name in data_layer_names:
    # Apply valid mask (calculated above) and good pixel mask with scale and offset
    layer = data[[layer_name]].where(valid_mask[layer_name] & good_pixel_mask) * scale
    rs.append(layer)
    
result = xr.merge(rs).persist()  # Calculate intermediate result
result  # Type: float32

## Calculate NDVI

Two options:
- `calculate_indices()` - a large set of Indices available. May change dataset variable names if satellite_mission is landsat or sentinel-2 (required).
- band math - calculate manually, if `calculate_indices()` is not suitable.

In [ ]:
# Frequency sampling codes - https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases

ds1 = calculate_indices(result, index='NDVI', satellite_mission='s2')
ndvi = ds1["NDVI"]
average_ndvi = ndvi.resample(time='1M').mean().persist()  ## tính mean cho từng tháng -> time = 12
average_ndvi  # DataArray

In [ ]:
# A standard matplotlib picture
# Will struggle for large datasets and may kill the kernel (out of memory)
# Creating an image will trigger processing of the dask tasks. Open the dask dashboard to see progress.

average_ndvi.plot(col="time", size=5, clim=(0,1), cmap="RdYlGn")

In [ ]:
# An interactive holoviz plot
# Works for large datasets with dask
# An image of the data is created on-demand (see https://datashader.org/getting_started/Pipeline.html)

xx = ndvi.to_dataset().hvplot(
    groupby='time',rasterize=True,
    cmap="RdYlGn", clim=(0,1),
    height=500,
)
xx

## Extract training points from the average NDVI dataset

*Note:* This section has not been fully check/tested (by Matt) yet. May need to filter `NaN` values from `load_datasets`.

In [ ]:
loaded_datasets = {}
for idx, point in train.iterrows():
    key = f"point_{idx + 1}"
    try:
        loaded_datasets[key] = {
            "NDVI": average_ndvi.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values,
            "label": point.Name
                               }
    except Exception as e:
        # loaded_datasets[key] = None
        print(key)

In [ ]:
loaded_datasets

In [ ]:
## tiền xử lý data: fill nan, remove 

In [ ]:
label_encoder = LabelEncoder()

# Fit and transform the labels
labels = train.Name.values
numeric_labels = label_encoder.fit_transform(labels)
label_mapping = dict(zip(labels, numeric_labels))

In [ ]:
loaded_datasets

In [ ]:
X = []
for k, v in loaded_datasets.items():
    X.append(v)

In [ ]:
x_new = []
lb_new = []
for i in range(len(X)):
    if X[i] is not None:
        x_new.append(X[i])
        lb_new.append(numeric_labels[i])

In [ ]:
df = pd.DataFrame(x_new)

In [ ]:
column_means = np.nanmean(x_new, axis=0)
column_means_expanded = np.tile(column_means, (len(x_new), 1))

In [ ]:
x_final = np.where(np.isnan(x_new), column_means_expanded, x_new)

In [ ]:
x_final

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_final, lb_new, test_size=0.3, random_state=42)


In [ ]:
model = RandomForestClassifier(n_estimators=1000, random_state=42, criterion='gini', max_depth=10)
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Tạo RandomForestClassifier mặc định để sử dụng làm mô hình ban đầu trong pipeline
base_model = RandomForestClassifier(random_state=42)

# Tạo pipeline
pipeline = Pipeline([
    # ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', base_model),
])

# Thiết lập các tham số bạn muốn tối ưu hóa
param_grid = {
    'classifier__n_estimators': [100, 500, 1000],
    'classifier__max_depth': [5, 10, 20],
    'classifier__criterion': ['gini', 'entropy'],
}

# Sử dụng GridSearchCV để tìm bộ tham số tốt nhất
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# In ra bộ tham số tốt nhất
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Dự đoán trên tập kiểm tra
y_pred = grid_search.predict(X_test)

# Đánh giá kết quả
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
avg = average_ndvi.persist()

In [ ]:
avg1 = avg.fillna(avg.mean(dim='x'))

In [ ]:
grid_search.predict([avg1.isel(y=0, x=0).values])

In [ ]:
client.close()
cluster.close()